In [0]:
import mlflow
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datetime import datetime, timezone
from scipy.stats import norm
from math import sqrt
import mlflow.pyfunc
from mlflow.tracking import MlflowClient

from io import BytesIO
from azure.storage.blob import BlobServiceClient

## 실시간 추론 모델 클래스

In [0]:
import pandas as pd
from sqlalchemy import create_engine

class RealtimeRealScoreModel(mlflow.pyfunc.PythonModel):

    def load_context(self, context):
        import mlflow.sklearn
        self.model = mlflow.sklearn.load_model(context.artifacts["model"])

    def fetch_bronze_question(self):
        blob_connection_string = "DefaultEndpointsProtocol=https;AccountName=team4mlblob;AccountKey=qU3qjqdPjn/LlGZzIfI/ox6zVb6BhIo1Dn1PRr4akHTJLlpQ9x8rHbtZsRFvTCgjh5Qpn/4td+q3+AStbBi+PQ==;EndpointSuffix=core.windows.net"
        container_name = "blobml"
        blob_name = "bronze_questions.csv"

        blob_service_client = BlobServiceClient.from_connection_string(blob_connection_string)
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
        download_stream = blob_client.download_blob()
        df_bronze = pd.read_csv(BytesIO(download_stream.readall()))
        
        return df_bronze

    def prepare_input(self, df_input):
        """입력 df와 bronze_question 조인 (input 우선, _x/_y 제거)"""
        df_bronze = self.fetch_bronze_question()

        if not df_input.empty and not df_bronze.empty:
            join_cols = ["testID", "assessmentItemID"]
            for col in join_cols:
                if col in df_input.columns and col in df_bronze.columns:
                    df_input[col] = df_input[col].astype(str)
                    df_bronze[col] = df_bronze[col].astype(str)

            # merge (기본 _x, _y suffix 사용)
            df_merged = pd.merge(df_input, df_bronze, on=join_cols, how="left")

            # 입력값 우선으로 모든 겹치는 컬럼 처리
            # 먼저 _x 컬럼들을 처리
            for col in list(df_merged.columns):
                if col.endswith('_x'):
                    base_col = col[:-2]  # '_x' 제거
                    bronze_col = f"{base_col}_y"
                    
                    if bronze_col in df_merged.columns:
                        # 입력값 우선으로 결합 (입력값이 없을 때만 bronze 값 사용)
                        df_merged[base_col] = df_merged[col].combine_first(df_merged[bronze_col])
                    else:
                        # bronze에 없는 컬럼은 그냥 원래 이름으로 변경
                        df_merged[base_col] = df_merged[col]
            
            # 이제 _x, _y suffix 컬럼들을 모두 삭제
            suffix_cols = [col for col in df_merged.columns if col.endswith('_x') or col.endswith('_y')]
            df_merged = df_merged.drop(columns=suffix_cols)

        else:
            df_merged = df_bronze.copy()

        # 결측치 처리 및 숫자 변환
        required_cols = ['is_correct', 'grade', 'gender', 'discriminationLevel', 'difficultyLevel', 'guessLevel']
        df_merged = df_merged.reindex(columns=df_merged.columns.union(required_cols))

        for col in required_cols:
            df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

        return df_merged
    
    # theta 추정
    def estimate_theta(self, a_list, b_list, c_list, y_list, grid_n=161):
        # (원래 코드 그대로)
        import numpy as np
        from scipy.optimize import minimize
        from scipy.stats import norm
        from math import sqrt

        def p_3pl(theta, a, b, c):
            z = np.outer(theta, a) - a * b
            sigmoid = 1 / (1 + np.exp(-z))
            return c + (1 - c) * sigmoid

        def neg_loglike(th):
            probs = p_3pl(np.array([th[0]]), a_list, b_list, c_list)[0]
            probs = np.clip(probs, 1e-12, 1-1e-12)
            ll = y_list * np.log(probs) + (1-y_list) * np.log(1-probs)
            return -np.sum(ll)

        try:
            res = minimize(neg_loglike, x0=[0.0], bounds=[(-4.0, 4.0)])
            theta_mle = float(res.x[0]) if res.success else np.nan
        except:
            theta_mle = np.nan

        theta_grid = np.linspace(-4, 4, grid_n)
        probs_grid = p_3pl(theta_grid, a_list, b_list, c_list)
        probs_grid = np.clip(probs_grid, 1e-12, 1-1e-12)
        log_lik = np.sum(y_list*np.log(probs_grid) + (1-y_list)*np.log(1-probs_grid), axis=1)
        log_prior = norm.logpdf(theta_grid, 0, 1)
        log_post = log_lik + log_prior
        post = np.exp(log_post - np.max(log_post))
        post /= post.sum()

        theta_eap = float((post * theta_grid).sum())
        theta_sd = sqrt(((theta_grid - theta_eap)**2 * post).sum())
        expected_score = float(p_3pl([theta_eap], a_list, b_list, c_list)[0].sum())
        return theta_mle, theta_eap, theta_sd, expected_score

    def calculate_theta_features(self, df_input):
        results = []
        if len(df_input) > 0:
            for (learner, test), g in df_input.groupby(['learnerID', 'testID']):
                a_list = g['discriminationLevel'].values
                b_list = g['difficultyLevel'].values
                c_list = g['guessLevel'].values
                y_list = g['is_correct'].astype(int).values
                
                theta_mle, theta_eap, theta_sd, expected_score = self.estimate_theta(a_list, b_list, c_list, y_list)
                
                results.append({
                    'learnerID': learner,
                    'gender': g['gender'].iloc[0],
                    'grade': g['grade'].iloc[0],
                    'testID': test,
                    'theta_clean': theta_eap,
                    'theta_sd': theta_sd,
                    'difficultyLevel': g['difficultyLevel'].mean(),
                    'discriminationLevel': g['discriminationLevel'].mean(),
                    'guessLevel': g['guessLevel'].mean(),
                    'correct_cnt': int(y_list.sum()),
                    'items_attempted': len(y_list),
                    'expected_score': expected_score
                })
        else:
            results.append({k: None for k in [
                'learnerID', 'gender', 'grade', 'testID', 'theta_clean', 'theta_sd',
                'difficultyLevel', 'discriminationLevel', 'guessLevel',
                'correct_cnt', 'items_attempted', 'expected_score'
            ]})
        
        df_features = pd.DataFrame(results)
        df_features['accuracy'] = df_features['correct_cnt'].fillna(0) / df_features['items_attempted'].replace(0, 1)
        return df_features

    def predict(self, context, model_input: pd.DataFrame):
        # 상태 코드 초기화
        status_code = 200
        error_category = ""

        # 1️⃣ 필수 컬럼 체크
        required_cols = ['is_correct', 'grade', 'gender', 'discriminationLevel', 'difficultyLevel', 'guessLevel']
        missing_cols = [c for c in required_cols if c not in model_input.columns]
        if missing_cols:
            status_code = 400
            error_category = f"Missing columns: {missing_cols}"
            raise ValueError(error_category)  # HTTP 400로 반환

        # 2️⃣ 타입/범위 체크
        if not pd.api.types.is_numeric_dtype(model_input['grade']):
            status_code = 422
            error_category = "grade 컬럼 타입 오류"
            raise TypeError(error_category)  # HTTP 422로 반환

        # 3️⃣ 정상 처리
        df_merged = self.prepare_input(model_input)
        df_features = self.calculate_theta_features(df_merged)

        feature_columns = ['theta_clean', 'accuracy', 'grade', 'gender',
                        'difficultyLevel', 'discriminationLevel', 'guessLevel']
        for col in feature_columns:
            df_features[col] = pd.to_numeric(df_features.get(col, 0), errors='coerce').fillna(0)

        if not df_features.empty:
            features_for_prediction = df_features[feature_columns]
            df_features['realScore_clean'] = self.model.predict(features_for_prediction)
        else:
            df_features['realScore_clean'] = 0

        # ✅ 로그는 따로 처리하므로 여기서는 raise만 사용
        return df_features

## 실시간 추론 서비스 배포

In [0]:
# MLflow 설정
EXPERIMENT_NAME = "/Users/1dt003@msacademy.msai.kr/team4_inference_experiment"
REALTIME_MODEL_NAME = "realtimeinference-model"
BASE_MODEL_NAME = "team4-pred-models"  # 기존에 저장된 모델명

mlflow.set_experiment(EXPERIMENT_NAME)
client = MlflowClient()

print("=== 실시간 추론 모델 배포 시작 ===")

# 기존 저장된 모델의 최신 버전 정보 가져오기
try:
    base_model_version = client.get_latest_versions(BASE_MODEL_NAME, stages=["None"])[0]
    base_model_uri = f"models:/{BASE_MODEL_NAME}/{base_model_version.version}"
    print(f"기존 모델 로드: {base_model_uri}")
except Exception as e:
    print(f"기존 모델 로드 실패: {e}")

# 실시간 추론 모델 배포
with mlflow.start_run(run_name="realtime_inference_deployment") as run:
    
    # 실시간 추론 모델을 MLflow에 등록
    mlflow.pyfunc.log_model(
        artifact_path="realtime_models",
        python_model=RealtimeRealScoreModel(),
        artifacts={"model": base_model_uri},  # 기존 모델을 artifact로 포함
        registered_model_name=REALTIME_MODEL_NAME,
        pip_requirements=[
            "mlflow",
            "pandas",
            "numpy",
            "scipy",
            "scikit-learn",
            "azure-storage-blob"
        ]
    )
    
    realtime_run_id = run.info.run_id
    print(f"실시간 모델 배포 완료. Run ID: {realtime_run_id}")


## 모델 서빙 준비 & 테스트

In [0]:
# 배포된 모델 로드
model_uri = f"models:/{REALTIME_MODEL_NAME}/latest"
print(f"모델 로드 중: {model_uri}")

try:
    loaded_model = mlflow.pyfunc.load_model(model_uri)
    print("모델 로드 성공!")
except Exception as e:
    print(f"모델 로드 실패: {e}")
    # 대안: run_id 직접 사용
    model_uri = f"runs:/{realtime_run_id}/realtime_model"
    loaded_model = mlflow.pyfunc.load_model(model_uri)
    print("Run ID로 모델 로드 성공!")

## 테스트 데이터 준비 및 추론 테스트

In [0]:

print("\n=== 실시간 추론 테스트 시작 ===")

# 테스트 데이터 1: 기본 케이스
test_input_1 = pd.DataFrame({
    'learnerID': [1, 1, 1, 2, 2, 2],
    'testID': [100, 100, 100, 100, 100, 100],
    'assessmentItemID': [101, 102, 103, 101, 102, 103],
    'discriminationLevel': [1.2, 0.8, 1.5, 1.2, 0.8, 1.5],
    'difficultyLevel': [0.5, -0.2, 1.1, 0.5, -0.2, 1.1],
    'guessLevel': [0.1, 0.15, 0.2, 0.1, 0.15, 0.2],
    'is_correct': [1, 0, 1, 0, 1, 0],
    'grade': [9, 9, 9, 10, 10, 10],
    'gender': ['M', 'M', 'M', 'F', 'F', 'F']
})

# 테스트 데이터 2: 다른 학습자
test_input_2 = pd.DataFrame({
    'learnerID': [3, 3, 3, 4, 4],
    'testID': [101, 101, 101, 101, 101],
    'assessmentItemID': [201, 202, 203, 201, 202],
    'discriminationLevel': [1.0, 1.3, 0.9, 1.0, 1.3],
    'difficultyLevel': [0.2, 0.8, -0.1, 0.2, 0.8],
    'guessLevel': [0.12, 0.18, 0.25, 0.12, 0.18],
    'is_correct': [1, 1, 0, 1, 0],
    'grade': [11, 11, 11, 12, 12],
    'gender': ['M', 'M', 'M', 'F', 'F']
})

# 테스트 실행
test_cases = [
    ("기본 테스트 케이스", test_input_1),
    ("다른 학습자 테스트", test_input_2)
]

for test_name, test_data in test_cases:
    print(f"\n--- {test_name} ---")
    print("입력 데이터:")
    print(test_data.head())
    
    try:
        # 추론 실행
        prediction = loaded_model.predict(test_data)
        
        print("예측 결과:")
        if 'error' in prediction.columns:
            print(f"오류 발생: {prediction['error'].iloc[0]}")
        else:
            result_columns = ['learnerID', 'testID', 'theta_clean', 'accuracy', 'realScore_clean']
            available_columns = [col for col in result_columns if col in prediction.columns]
            print(prediction[available_columns])
            
    except Exception as e:
        print(f"추론 실행 중 오류: {str(e)}")


# 성능 테스트 (응답 시간)
print("\n=== 성능 테스트 ===")
import time

# 응답 시간 측정
response_times = []
for i in range(5):
    start_time = time.time()
    try:
        prediction = loaded_model.predict(test_input_1)
        end_time = time.time()
        response_time = end_time - start_time
        response_times.append(response_time)
        print(f"테스트 {i+1}: {response_time:.3f}초")
    except Exception as e:
        print(f"테스트 {i+1} 실패: {str(e)}")

if response_times:
    avg_response_time = np.mean(response_times)
    print(f"평균 응답 시간: {avg_response_time:.3f}초")
    print(f"최소 응답 시간: {min(response_times):.3f}초")
    print(f"최대 응답 시간: {max(response_times):.3f}초")

print("\n=== 실시간 추론 서비스 배포 및 테스트 완료 ===")

mlflow models serve -m "models:/realtime-realScore-model/latest" -p 1234 --no-conda